In [3]:
import coinbasepro
import time
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import plotly

import  plotly.plotly as plt
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *

#plotly.tools.set_credentials_file(username='xxx', api_key='zzzz') You will set your own plotly API keys here

In [4]:
def moving_average(df, n):
    """Calculate the moving average for the given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    Source: https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py
    """
    MA = pd.Series(df['close'].rolling(n, min_periods=n).mean(), name='{}_SMA'.format(n))
    df = df.join(MA)
    return df

def exponential_moving_average(df, n):
    """
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    Source: https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py
    """
    EMA = pd.Series(df['close'].ewm(span=n, min_periods=n).mean(), name= '{}_EMA'.format(n))
    df = df.join(EMA)
    return df

def relative_strength_index(df, n):
    """Calculate Relative Strength Index(RSI) for given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    Source: https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py
    """
    i = 0
    UpI = [0]
    DoI = [0]
    while i + 1 <= df.index[-1]:
        UpMove = df.loc[i + 1, 'high'] - df.loc[i, 'high']
        DoMove = df.loc[i, 'low'] - df.loc[i + 1, 'low']
        if UpMove > DoMove and UpMove > 0:
            UpD = UpMove
        else:
            UpD = 0
        UpI.append(UpD)
        if DoMove > UpMove and DoMove > 0:
            DoD = DoMove
        else:
            DoD = 0
        DoI.append(DoD)
        i = i + 1
    UpI = pd.Series(UpI)
    DoI = pd.Series(DoI)
    PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
    NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
    RSI = pd.Series(PosDI / (PosDI + NegDI), name='{}_RSI'.format(n))
    df = df.join(RSI)
    return df

def bbands(df, period = 4, num_of_std=2):
    """Utilize Bollinger bands to set a cut-loss & sell price
    
    :param df: pandas.DataFrame
    :param period: Period for BBands
    :param num_of_std: Number of standard deviations
    :return: pandas.DataFrame
    Source: https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py
    """
    rolling_mean = pd.Series(df['close'].rolling(period, min_periods=period).mean())
    rolling_stdv = pd.Series(df['close'].rolling(period, min_periods=period).std())
    upper = rolling_mean + (rolling_stdv* num_of_std)
    B1 = pd.Series(upper, name='BBands_up_{}'.format(str(period)))
    df = df.join(B1)
    lower = rolling_mean - (rolling_stdv*num_of_std)
    B2 = pd.Series(lower, name='BBands_low_{}'.format(str(period)))
    df = df.join(B2) 
    return df

def add_line(df, s_name, l_color):
    a = Scatter(x=df['time'],
                y=df[s_name],
                name=s_name,
                line=scatter.Line(color=l_color))
    return a

In [5]:
p_client = coinbasepro.PublicClient()

In [6]:
time_now_iso = datetime.now().isoformat()

In [7]:
#Gets past price history. Format of time returned is EPOCH TIME
history_300 = sterm_builder(p_client.get_product_historic_rates(product_id = 'ETH-USD', start=time_now_iso, granularity = 300))
history_900 = lterm_builder(p_client.get_product_historic_rates(product_id = 'ETH-USD', start=time_now_iso, granularity = 900))
history_3600 = lterm_builder(p_client.get_product_historic_rates(product_id = 'ETH-USD', start=time_now_iso, granularity = 3600))

In [8]:
#history_300.to_excel("5_min.xlsx")
history_900.to_excel("15_min.xlsx")
history_3600.to_excel("1_hr.xlsx")

In [9]:
#Visualize via candle and line plot
candle_df = history_3600

trace = Candlestick(x=candle_df['time'],
                    open=candle_df['open'],
                    high=candle_df['high'],
                    low=candle_df['low'],
                    close=candle_df['close'])

data = [trace]

plt.iplot(data, filename='simple_candlestick')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~khoo-j/0 or inside your plot.ly account where it is named 'simple_candlestick'


In [10]:
#Visualize the moving averages
dict_lines= (#{'name': '5_SMA','color':'rgb(0,255,0)'},
             #{'name':'8_SMA','color':'rgb(0,128,128)'},
             #{'name':'13_SMA','color':'rgb(0,0,255)'},
             #{'name':'5_EMA','color':'rgb(0,255,255)'},
             #{'name':'8_EMA','color':'rgb(255,0,255)'},
             #{'name':'13_EMA','color':'rgb(255,0,0)'},
             {'name':'BBands_up_4','color':'rgb(244, 66, 215)'},
             {'name':'BBands_low_4','color':'rgb(244, 66, 215)'},
             )

sma_plot = []
ema_plot = []

for line in dict_lines:
    scatter_output = add_line(candle_df, line['name'], line['color'])
    if 'SMA' in line['name']:
        sma_plot.append(scatter_output)
    else:
        ema_plot.append(scatter_output)
        
plt.iplot(ema_plot, filename = 'scatter-eth')